In [ ]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages
    !pip install ase torch_geometric
    import torch
    from torch_geometric.data import Data, DataLoader

    dataset = "/content/drive/My Drive/Dataset"
else:
    dataset = "Dataset"

In [ ]:
import pandas as pd
import numpy as np
import scipy

In [ ]:
path_pkl = f"{dataset}/absorption_mp_data.pkl"
pkl_df = pd.read_pickle(path_pkl)
pkl_df.head()

In [ ]:
sample_structure = np.array(pkl_df["structure"][0])
sample_structure

## Krammer-Kronig's relation

$\epsilon_1(\omega)$ = real part of the dielectric function

$\epsilon_2(\omega)$ = imaginary part of the dielectric funcion

$\alpha (\omega)$ = absorption coefficient

$n(\omega)$ = real part of the refractive index

$k(\omega)$ = imaginary part of the refractive index

$R(\omega)$ = reflectance

$\omega$ = angular frequency = $\frac{\text{energy}}{\hbar}$

In [ ]:
print(scipy.constants.Planck)
print(scipy.constants.physical_constants["electron volt-joule relationship"][0])
print(scipy.constants.pi)
print(scipy.constants.c)


In [ ]:
import scipy


class Krammer_Kronig:
    def __init__(self, real_dielectric, imag_dielectric, photon_energy):
        self.real_dielectric = real_dielectric
        self.imag_dielectric = imag_dielectric
        self.photon_energy = photon_energy * scipy.constants.physical_constants["electron volt-joule relationship"][0]
        self.reduced_plank = scipy.constants.Planck/(2*scipy.constants.pi)
        self.ang_frequency = self.photon_energy/self.reduced_plank

    
    def get_absorption_coff(self):
        left= (np.sqrt(2)*self.ang_frequency)/(scipy.constants.c)
        first_square_root = np.sqrt(np.square(self.real_dielectric) + np.square(self.imag_dielectric))
        right = np.sqrt(first_square_root - self.real_dielectric)
        # right = np.sqrt(np.sqrt(np.square(self.real_dielectric) + np.square(self.imag_dielectric)) - self.real_dielectric)
        self.absorption_coff = left * right
        return self.absorption_coff

    # Since we can get n, k, and R from the real and iaginary parts of the dielectric...    
    def get_real_refractive_index(self):
        self.real_refractive = np.sqrt(0.5*(np.sqrt(np.square(self.real_dielectric)+np.square(self.imag_dielectric))+self.real_dielectric))
        return self.real_refractive

    def get_imag_refractiv_index(self):
        self.imag_refractive = np.sqrt(0.5*(np.sqrt(np.square(self.real_dielectric)+np.square(self.imag_dielectric))-self.real_dielectric))
        return self.imag_refractive

    def get_reflectance(self):
        num = np.square(self.real_refractive - 1) + np.squar(self.imag_refractive)
        den = np.square(self.real_refractive + 1) + np.squar(self.imag_refractive)
        self.reflectance = num/den
        return self.reflectance


In [ ]:
# Get absorption coefficient
sample_real = pkl_df["real_dielectric"][0][2]
sample_imag = pkl_df["imag_dielectric"][0][2]
sample_energy = pkl_df["energies"][0][2]

print(sample_real)
print(sample_imag)
print(sample_energy)

inst_1 = Krammer_Kronig(sample_real,sample_imag,sample_energy)
result_coff = inst_1.get_absorption_coff()

the_ref = pkl_df["absorption_coefficient"][0][2]

if result_coff == the_ref:
    print("calculations work perfectly")
else:
    print(f"Actual value: {the_ref}\nResult value: {result_coff}")



Why is the $\text{result value} = \text{actual value} *100$?